# Visualisation
installing new packages on metacebtrum: `/usr/bin/python3 -m pip install -U plotly`


In [1]:
import numpy as np
import plotly.graph_objects as go
import json

In [11]:
figure = go.Figure(
        layout=go.Layout(template="simple_white",
        xaxis=dict(title="Iteration", mirror=True, showline=True),
        yaxis=dict(title="Energy", mirror=True, showline=True),
        title=("Energy convergence")))

In [12]:
# import the data from log file
OUT_NAME_suffixless="../models16_RBM_2_0.01_"
DS_AF_switch = 0 # 0...DS, 1...AF

exact_ground_energy = [-24,-34.0819077860][DS_AF_switch]
OUT_NAME_suffixless = OUT_NAME_suffixless+["DS_","AF_"][DS_AF_switch]
figure.add_hline(y=exact_ground_energy, opacity=1, line_width=1)
no_of_runs = 2
data = []
for i in range(no_of_runs):
    data.append(json.load(open(OUT_NAME_suffixless+["normal","MSR"][i]+".log")))
names = ["normal","MSR"]
if type(data[0]["Energy"]["Mean"]) == dict: #DTYPE in (np.complex128, np.complex64):#, np.float64):# and False:
    energy_convergence = [data[i]["Energy"]["Mean"]["real"] for i in range(no_of_runs)]
    # symmetry = [data[i]["symmetry"]["Mean"]["real"] for i in range(no_of_runs-use_MSR)]
else:
    energy_convergence = [data[i]["Energy"]["Mean"] for i in range(no_of_runs)]
    # symmetry = [data[i]["symmetry"]["Mean"] for i in range(no_of_runs-use_MSR)]
for i in range(no_of_runs):
    figure.add_trace(go.Scatter(
        x=data[i]["Energy"]["iters"], y=energy_convergence[i],
        name=names[i]
    ))
    # figure.add_trace(go.Scatter(
    #     x=data[i]["Energy"]["iters"], y=symmetry[i],
    #     name=names[i]+"_swap"
    # ))

#figure.add_hline(y=exact_gs_energy)
figure.update_layout(xaxis_title="Iteration",yaxis_title="Energy")
figure.show()

### N=16 hyperparameter search

In [4]:
j1 = 0.9
OUT_NAME_suffixless="N=16_search/"
no_of_runs = 5
exact_energies = {0.1: -24, 0.3: -24, 0.5: -24, 0.7: -25.4010078362, 0.9:-34.0819077860}

figure = go.Figure(
        layout=go.Layout(template="simple_white",
        xaxis=dict(title="Iteration", mirror=True, showline=True),
        yaxis=dict(title="Energy", mirror=True, showline=True),
        title=("Energy convergence")))
data = []
for i in range(no_of_runs):
    data.append(json.load(open(OUT_NAME_suffixless+str(i+1)+"/RBM16_"+str(j1)+"_0.log")))
names = ["η=0.010,α=16","η=0.010,α=08","S=0,η=0.01,α=16","η=0.002,α=16","η=0.100,α=16",]

if type(data[0]["Energy"]["Mean"]) == dict: #DTYPE in (np.complex128, np.complex64):#, np.float64):# and False:
    energy_convergence = [data[i]["Energy"]["Mean"]["real"] for i in range(no_of_runs)]
else:
    energy_convergence = [data[i]["Energy"]["Mean"] for i in range(no_of_runs)]
for i in range(no_of_runs):
    figure.add_trace(go.Scatter(x=data[i]["Energy"]["iters"], y=energy_convergence[i],name=names[i]))
figure.add_trace(go.Scatter(x=(0,3000),y=(exact_energies[j1],exact_energies[j1]),mode="lines",line=go.scatter.Line(color="#000000",width=1), name="exact energy"))

#figure.add_hline(y=exact_gs_energy)
figure.update_layout(xaxis_title="Iteration",yaxis_title="Energy",title="Energy convergence for J1="+str(j1))
figure.show()